In [70]:
library(xlsx)
options(warn=-1)
data_full <- read.xlsx("TARGET_AML_ClinicalData_20160714.xlsx",sheetIndex = 1, stringsAsFactors = FALSE)
dimnames(data_full)[[1]] <- data_full[,1]
dim(data_full)
expn <- read.csv("AML_assay_clinical.csv",header=TRUE)
dimnames(expn)[[1]] <- expn[,1]
expn <- expn[, -1]
expn = expn[1:108,]
dim(expn)

[1] 993  63

[1]   108 21491

In [2]:
binary_s2n<-function(mat,feature){
    efst=mat$Event.Free.Survival.Time.in.Days
    vec=mat[,feature]
    dirty=which((vec=="Unknown")|(vec=="Not done"))
    if (length(dirty)>0){
        vec=vec[-dirty]
        efst=efst[-dirty]}
    vec=as.numeric(as.factor(vec))
    pos=which(vec==1)
    neg=which(vec==2)
    pos_samp=efst[pos]
    neg_samp=efst[neg]
    m1=mean(pos_samp)
    m2=mean(neg_samp)
    sd1=sd(pos_samp)
    sd2=sd(neg_samp)
    return (abs(m1-m2)/(sd2+sd1))
}

In [3]:
binary_features=c('Gender','CNS.disease','Chloroma','t.6.9.','t.8.21.','t.3.5..q25.q34.','t.6.11..q27.q23.','t.9.11..p22.q23.','t.10.11..p11.2.q23.','t.11.19..q23.p13.1.','inv.16.','del5q','del7q','del9q','monosomy.7','trisomy.8','trisomy.21','MLL','Minus.Y','Minus.X')

In [4]:
features2n=lapply(binary_features,function(x){return(binary_s2n(data_full,x))})
binary_s2n(data_full,"monosomy.5")
features2n

[1] NA

[[1]]
[1] 0.03158361

[[2]]
[1] 0.02682833

[[3]]
[1] 0.1765849

[[4]]
[1] 0.04398401

[[5]]
[1] 0.2350849

[[6]]
[1] 0.6886987

[[7]]
[1] 0.3719036

[[8]]
[1] 0.1090879

[[9]]
[1] 0.2876704

[[10]]
[1] 0.01216542

[[11]]
[1] 0.08692631

[[12]]
[1] 0.2086184

[[13]]
[1] 0.02542727

[[14]]
[1] 0.05621113

[[15]]
[1] 0.3672975

[[16]]
[1] 0.05074236

[[17]]
[1] 0.05951823

[[18]]
[1] 0.02779571

[[19]]
[1] 0.270769

[[20]]
[1] 0.2582709

In [76]:
gene_names=colnames(expn[85:21488])
genescor=lapply(gene_names,function(x){return(cor(as.numeric(expn$Event.Free.Survival.Time.in.Days),exp(as.numeric(expn[,x])),use="complete.obs"))})

In [78]:
length(which(abs(as.numeric(genescor))>0.4))
genescor[which.max(abs(as.numeric(genescor)))]
gene_names[which.max(abs(as.numeric(genescor)))]

[1] 57

[[1]]
[1] 0.5141186

[1] "ENSG00000269743.2"

In [112]:
selected_genes=gene_names[which(abs(as.numeric(genescor))>0.4)]
expn_important=exp(expn[,selected_genes])
expn_important$Event.Free.Survival.Time.in.Days=expn$Event.Free.Survival.Time.in.Days

In [126]:
train_batch=expn_important[1:100,]
test_batch=expn_important[101:108,]
res=lm(Event.Free.Survival.Time.in.Days ~ ., data=train_batch)
bck=step(res,trace=0,direction=c("backward"))
#summary(bck)
test_res=predict(bck,newdata=test_batch)
test_res
test_batch$Event.Free.Survival.Time.in.Days

100        101        102        103        104        105        106 
1414.46779   89.02724  604.21585 1807.35169  903.16523  953.71403  123.18905 
       107 
1126.75945

[1] 2457  318 1798 2760  299  294  550 1199